In [184]:
from lib.csv_to_array import convert as pre_process

# George Frederick Handel (robot)

```
[0, 0, Header, format, nTracks, division]
[Track, 0, Start_track]
[Track, Time, Note_off_c, Channel, Note, Velocity]
```
checkout http://www.fourmilab.ch/webtools/midicsv/ for more info

In [185]:
gigue = './csv/handel_hwv-433_5_gigue_(c)yamada.mid.csv'
# gigue = './csv/total.csv'

ga = pre_process(gigue)

print "i.e: \n" + str(ga[32])

i.e: 
['1', ' 190', ' Note_on_c', ' 0', ' 68', ' 0']


In [186]:
#let's drop the time bit for now
whatevs = [g.pop(1) for g in ga]

In [187]:
#padding 
s_len = max([len(g) for g in ga])

p_ga = [g + [None] * (s_len - len(g)) for g in ga]

elems = ['track', 'action', 'channel', 'pitch', 'velocity', 'hurh?']

v = []
s = []
iv = []
l_v = []
d_l = len(p_ga)

for i in range(s_len):
    v.append([g[i] for g in p_ga])
    s.append(set(v[i]))
    iv.append([(ind, na) for ind, na in enumerate(s[i])])
    l_v.append(len(s[i]))
    print("there are fewer than " + str(l_v[i]) 
          + " unique " + elems[i] + " in piece with nearly "
          + str(d_l) + " events!")

ix_to_val = lambda y, f: [x for x in f if x[0] == y][0][1]
val_to_ix = lambda y, f: [x for x in f if x[1] == y][0][0]

there are fewer than 3 unique track in piece with nearly 2522 events!
there are fewer than 12 unique action in piece with nearly 2522 events!
there are fewer than 44 unique channel in piece with nearly 2522 events!
there are fewer than 48 unique pitch in piece with nearly 2522 events!
there are fewer than 8 unique velocity in piece with nearly 2522 events!
there are fewer than 3 unique hurh? in piece with nearly 2522 events!


Let's include crossover on the hidden levels - the last ones.
And we're going to ignore the time element until we can implement a 'sequence generation' network.

Open Questions:
 - does normalizing the length of each vocabulary make it easier to pure numpy arrays?
    - I kinda think numpy arrays in lists viz a viz list comprehensions is not such a bad thing
 - what about sharing?
    - my feeling is that each element will have len(elements) hidden weight matrices, and we will be applying them against the element's hidden layer -- do we need hidden layers, plural?
      my instinct is: no - we will be defining the hidden layer as:
      tanh(dot(in-wgts, in-vals) + dot(last-self-weights, last-self) + dot(last-other-weights, last-other) ... etc etc. This fits the analogy - we don't really need to prefer any previous values, but we DO want to be sensitive to temporal patterns in every dimension. Let's do this.

In [188]:
#the fun begins
import numpy as np

# hyperparameters
hidden_size = 100 # size of hidden layer of neurons
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 1e-1

# weights belong only to the individual network, values do not
# model parameters
weights = [{'Wxh':np.random.randn(hidden_size, l_v[i])*0.01,
            'Whh':np.random.randn(s_len, hidden_size, hidden_size)*0.01,
            'Why':np.random.randn(l_v[i], hidden_size)*0.01,
            'bh':np.zeros((s_len, hidden_size, 1)),
            'by':np.zeros((l_v[i], 1))} for i in range(s_len)] # input to hidden

In [190]:
#let's do the main loop next
n = 0 #iteration counter
p = 0 #position in `events` array, `nm` - more generally `p_ga`

m_weights = [{'mWxh':np.zeros_like(w['Wxh']),
              'mWhh':np.zeros_like(w['Whh']),
              'mWhy':np.zeros_like(w['Why']),
              'mbh':np.zeros_like(w['bh']),
              'mby':np.zeros_like(w['by'])} for w in weights]

#hm - does loss belong to each network individually?
smooth_loss = [-1 * np.log(1.0/l_v[i]) * seq_length for i in range(s_len)]

In [191]:
while True:
    #position is absolute - invariant across elements
    if p + seq_length + 1 >= d_l or n == 0: #beinning & end: training data
        #hprev = [np.zeros((hidden_size,1)) for i in range(s_len)] #marks the start/end of seq
        hprev = np.zeros((s_len, hidden_size,1))
        p = 0 #start at the very beginning
        
    #transform first chunk of data into indices
    inputs = [[val_to_ix(nn, iv[ind]) for nn in n[p:p+seq_length]] for ind,n in enumerate(v)]
    #transform shifted+1 chunk of data for target vals
    targets = [[val_to_ix(nn, iv[ind]) for nn in n[p+1:p+seq_length+1]] for ind,n in enumerate(v)]
    
    #run the current seq through the net, and fetch the gradients
    #not so pretty - let's not use this in favor of an array of np arrays - 
    #must be so since input/output lengths are variant
    #let's see what happens - and use plain old numpy with normalized 
    xs = [np.zeros((seq_length, l)) for l in l_v] # python voodoo for initializing these vals - would numpy arr work better?
    hs = [np.zeros((seq_length, hidden_size, hidden_size,1)) for l in l_v]
    ys = [np.zeros((seq_length, max_v)) for l in l_v] # output states
    ps = [np.zeros((seq_length, max_v)) for l in l_v] # probabilities
    
    #I wonder what it would look like to have a hs with len less than seq_length
    
    loss = [] + [0] * s_len#does each element experience loss separately? probably
    
    #forward pass!
    #ew this is gross. can we do it purely with numpy if they're different sizes?
    for j in xrange(s_len):
        for t in xrange(len(inputs)): #1 for each val in seq - remind u of rk4?
            xs[j][t,inputs[t]] = 1 #one is hot! note the cool use of indexing
            hs[j][t] = np.tanh(np.dot(weights[j]['Wxh'], xs[j][t]) + sum(
                    np.dot(weights[j]['Whh'][i], hs[j][t-1]) for i in range(s_len)) + weights[j]['bh'])
            ys[j][t] = np.dot(weights[j]['Why'], hs[j][t]) + weights[j]['by'] #unnormed log prob for next word
            ps[j][t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) #sigmoid those probs
            #apply loss - log linear
            loss[j] += -np.log(ps[j][t,targets[j][t]])
        
    #backward pass! - might want to split this when we've got parallelized
    #interconnected networks
    
    d_weights = [{'dWxh':np.zeros_like(w['Wxh']),
              'dWhh':np.zeros_like(w['Whh']),
              'dWhy':np.zeros_like(w['Why']),
              'dbh':np.zeros_like(w['bh']),
              'dby':np.zeros_like(w['by'])} for w in weights]
    
    dhnext = [np.zeros_like(h[0]) for h in hs]
    dhraw = []

    dy = [np.copy(p) for p in ps] # python voodoo for initializing these vals - would numpy arr work better?
    dh = [np.zeros_like(h) for h in hs]
    dx = [np.zeros_like(x) for x in xs] # output states
    
    #that's right, we backprop to every sequence we visit - t has power
    for j in xrange(s_len):
        for t in reversed(xrange(len(inputs))):
            dy[j][targets[t]] -= 1 #actual value 'applied' to space, will fix
            d_weights[j]['dWhy'] += np.dot(dy[j], hs[j][t].T) #bp into y weights
            d_weights[j]['dby'] += dy[j]
            dh[j] = np.dot(weights[j]['Why'].T, dy[j]) + dhnext[j] #bp into h
            dhraw.append((1-hs[j][t]*hs[j][t]) * dh[j]) #take bp and filter via tanh(u)`
            d_weights[j]['dbh'] += dhraw[j] #makes sense, the bias accums via the bp of output
            d_weights[j]['dWxh'] += np.dot(dhraw[j], xs[j][t].T) #bp hidden into inputs weights
            d_weights[j]['dWhh'] += np.dot(dhraw[j], hs[j][t-1].T) #bp into t-1 hidden state weights
            dhnext[j] = np.dot(d_weights[j]['Whh'].T, dhraw[j])
        for keys in d_weights[j]:
            np.clip(d_weights[j][key],-5,5,out=d_weights[j][key]) #clip to mitigate exploding grad

    smooth_loss = [ss * 0.999 + loss[i] * 0.001 for i,ss in smooth_loss] #wow, such voodoo
    
    if n % 100 == 0:
        print('iter %d, loss %f,' % (n, sl) for sl in smooth_loss) #see that loss shrinkin?
    
    #adagrad - let's make this thing it's own bit?!
    #seems like afterthought, Karpathy!
    #such a business, but gives us tuple:
    #(cur-weights, del-weights, mem-weights? wat r mem-weights?)
    for j in range(s_len):
        for param, dparam, mem in zip(['Wxh', 'Whh', 'Why', 'bh', 'by'],
                                      ['dWxh', 'dWhh', 'dWhy', 'dbh', 'dby'],
                                      ['mWxh', 'mWhh', 'mWhy', 'mbh', 'mby']):
            m_weights[j][mem] += d_weights[j][dparam] * d_weights[j][dparam] #square delta weights into memory? y square?
            weights[j][param] += -learning_rate * d_weights[j][dparam] / np.sqrt(m_weights[j][mem] + 1e-8) #adagrad!
        
    p += seq_length
    n += 1 
    #let's go again!

ValueError: could not broadcast input array from shape (6,100,100) into shape (100,1)

In [ ]:
#generator - basically a forward only run
#   """ 
#   sample a sequence of integers from the model 
#   h is memory state, seed_ix is seed letter for first time step
#   """
def sample(h, seed_ix, n):
    x = np.zeros((vocab_size,1))
    x[seed_ix] = 1 #builds initial input - catalyst
    ixes = [] #sequence out!
    #forward pass, tidy loop
    for t in xrange(n):
        h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
        y = np.dot(Why, h) + by
        p = np.exp(y) / np.sum(np.exp(y))
        #p= option sets the probabilities of 'random' choice
        ix = np.random.choice(range(vocab_size), p=p.ravel())
        x = np.zeros((vocab_size, 1))
        x[ix] = 1
        ixes.append(ix)
    return ixes

In [ ]:
sample_ix = sample(hprev, inputs[0], 3500)
sample_notes = [note_for_ind(i)[1] for i in sample_ix]
print sample_notes